In [71]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import requests

In [14]:
# Scrapping for List of stocks in S&P500
url = input('Enter Wikipedia url for list of S&P500 Companies_')
  
wiki_data = pd.read_html(url)
print(wiki_data)

Enter Wikipedia url for list of S&P500 Companies_ https://en.wikipedia.org/wiki/List_of_S%26P_500_companies


[    Symbol              Security             GICS Sector  \
0      MMM                    3M             Industrials   
1      AOS           A. O. Smith             Industrials   
2      ABT                Abbott             Health Care   
3     ABBV                AbbVie             Health Care   
4      ACN             Accenture  Information Technology   
..     ...                   ...                     ...   
498    YUM           Yum! Brands  Consumer Discretionary   
499   ZBRA    Zebra Technologies  Information Technology   
500    ZBH         Zimmer Biomet             Health Care   
501   ZION  Zions Bancorporation              Financials   
502    ZTS                Zoetis             Health Care   

                      GICS Sub-Industry    Headquarters Location  Date added  \
0              Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1                     Building Products     Milwaukee, Wisconsin  2017-07-26   
2                 Health Care Equipmen

In [15]:
# Extracting Stock Price Historical Data from Yahoo! Finance using Tickers from Wikipedia
import yfinance as yf
from pandas.tseries.offsets import DateOffset
from datetime import datetime

ticker_df = wiki_data[0]
display(ticker_df.head(2))
display(ticker_df.tail(2))

def extract_stock_data():
    ticker_list= ticker_df['Symbol'].to_list()
    stock_list = ticker_df['Security'].to_list()
    print(stock_list)
    stock_name = input('Enter Stock_Name from the above displayed list for Analysis:')
    start_date = input('Enter Start Date for Historical Data in yyyy-mm-dd:')
    today = datetime.today().strftime('%Y-%m-%d')    
    row_index = ticker_df.query(f"Security == '{stock_name}'").index[0]
    stock_ticker = ticker_list[row_index]
    stock_df = yf.download(stock_ticker,start_date,today)
    stock_df.index = pd.to_datetime(stock_df.index)
    print(f"{stock_ticker}:{stock_name}")
    return(stock_df)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873
502,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


In [77]:
# Calling Specific Company stock data 
# Select Stock & Historical Date for Analysis
# using extract_stock_data() function

df = extract_stock_data()

display(df.head(3))
display(df.tail(3))

['3M', 'A. O. Smith', 'Abbott', 'AbbVie', 'Accenture', 'Adobe Inc.', 'Advanced Micro Devices', 'AES Corporation', 'Aflac', 'Agilent Technologies', 'Air Products and Chemicals', 'Airbnb', 'Akamai', 'Albemarle Corporation', 'Alexandria Real Estate Equities', 'Align Technology', 'Allegion', 'Alliant Energy', 'Allstate', 'Alphabet Inc. (Class A)', 'Alphabet Inc. (Class C)', 'Altria', 'Amazon', 'Amcor', 'Ameren', 'American Airlines Group', 'American Electric Power', 'American Express', 'American International Group', 'American Tower', 'American Water Works', 'Ameriprise Financial', 'Ametek', 'Amgen', 'Amphenol', 'Analog Devices', 'Ansys', 'Aon', 'APA Corporation', 'Apple Inc.', 'Applied Materials', 'Aptiv', 'Arch Capital Group', 'Archer-Daniels-Midland', 'Arista Networks', 'Arthur J. Gallagher & Co.', 'Assurant', 'AT&T', 'Atmos Energy', 'Autodesk', 'Automated Data Processing', 'AutoZone', 'AvalonBay Communities', 'Avery Dennison', 'Axon Enterprise', 'Baker Hughes', 'Ball Corporation', 'Bank

Enter Stock_Name from the above displayed list for Analysis: Union Pacific Corporation
Enter Start Date for Historical Data in yyyy-mm-dd: 2011-03-18


[*********************100%%**********************]  1 of 1 completed

UNP:Union Pacific Corporation


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-03-18,47.575001,48.000000,47.070000,47.509998,35.833424,9018800
2011-03-21,47.900002,48.470001,47.674999,48.285000,36.417950,3865400
2011-03-22,48.099998,48.430000,47.174999,47.314999,35.686352,6370400


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-03-13,250.259995,250.779999,248.460007,249.250000,249.250000,2129000
2024-03-14,247.429993,247.979996,243.600006,246.059998,246.059998,2519700
2024-03-15,243.449997,246.380005,243.229996,244.130005,244.130005,3258900


In [78]:
# Feature Engineering _ additionals features for ML Analysis

import pandas_ta as ta

def technical_indicators(OHLCV_df):
    # Daily Avg Price
    OHLCV_df['avg_price'] = OHLCV_df.ta.ohlc4()
    # Daily Returns
    OHLCV_df['daily_returns'] = OHLCV_df['Close'].pct_change()
    # 21Day Volatility
    OHLCV_df['volatility_21'] = OHLCV_df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    OHLCV_df['sma_21'] = OHLCV_df.ta.sma(length=21)
    OHLCV_df['sma_50'] = OHLCV_df.ta.sma(length=50)
    # Exponential Moving Avg 21Days & 50Days
    OHLCV_df['ema_21'] = OHLCV_df.ta.ema(length=21)
    OHLCV_df['ema_50'] = OHLCV_df.ta.ema(length=50)
    # VWAP modified to use Annual cummulative sum
    OHLCV_df['typical_price'] = (OHLCV_df['High']+OHLCV_df['Low']+OHLCV_df['Close'])/3
    OHLCV_df['vw_typical_price'] = OHLCV_df['typical_price']*OHLCV_df['Volume']
    OHLCV_df['vwap_annual'] = OHLCV_df.groupby(OHLCV_df.index.year)['vw_typical_price'].cumsum() / OHLCV_df.groupby(OHLCV_df.index.year)['Volume'].cumsum()
    # RSI Momemtum Indicator 14Days
    OHLCV_df['rsi_14'] = OHLCV_df.ta.rsi()
    # MACD Momentum Indicator
    OHLCV_df[['ema_12', 'ema_26', 'MACD_signal']] = OHLCV_df.ta.macd()
    # Bolinger Bands %B Indicator
    OHLCV_df[['BB_lower','BB_middle','BB_upper','BBP','BB%B']]= OHLCV_df.ta.bbands()
    # On-Balance Volume in Millions
    OHLCV_df['OBV_in_million'] = OHLCV_df.ta.obv()/1000000
    # Average Directional Index 14Days
    OHLCV_df[['adx_14','dmp_14','dmn_14']] = OHLCV_df.ta.adx()
    # Dropping trivial Columns
    OHLCV_append_df = OHLCV_df.drop(columns = ['typical_price','vw_typical_price',
                                               'BBP','dmp_14','dmn_14']).dropna()
    return(OHLCV_append_df)
   

In [79]:
# Extract Technical Indicators & Feature Engineering
# using technical_indicators function

df = technical_indicators(df)

df.tail()

,Open,High,Low,Close,Adj Close,Volume,avg_price,daily_returns,volatility_21,sma_21,...,rsi_14,ema_12,ema_26,MACD_signal,BB_lower,BB_middle,BB_upper,BB%B,OBV_in_million,adx_14
Date,,,,,,,,,,,,,,,,,,,,,
2024-03-11,250.220001,251.479996,247.289993,249.869995,249.869995,1539600,249.714996,-0.004343,2.942407,251.153333,...,50.282001,1.463303,-0.753955,2.217258,247.767808,250.913998,254.060188,0.334085,259.6658,15.571718
2024-03-12,249.419998,252.179993,249.220001,251.179993,251.179993,1633900,250.499996,0.005243,2.916436,251.235713,...,53.024318,1.340817,-0.701153,2.041970,248.757843,250.409998,252.062152,0.733028,261.2997,14.815413
2024-03-13,250.259995,250.779999,248.460007,249.250000,249.250000,2129000,249.687500,-0.007684,2.821164,251.323809,...,48.757398,1.075613,-0.773085,1.848698,248.317124,250.059998,251.802871,0.267626,259.1707,14.368420
2024-03-14,247.429993,247.979996,243.600006,246.059998,246.059998,2519700,246.267498,-0.012798,2.776957,251.345237,...,42.648496,0.601101,-0.998078,1.599179,245.778293,249.463998,253.149704,0.038216,256.6510,15.297208
2024-03-15,243.449997,246.380005,243.229996,244.130005,244.130005,3258900,244.297501,-0.007844,3.186383,251.060476,...,39.429686,0.068523,-1.224525,1.293048,242.893063,248.097998,253.302933,0.118824,253.3921,16.245715


In [80]:
# Preparing Features & Target DataFrame for ML Analysis

def predictors_target(stock_df):
    # Previous Day's Data as predictors
    #predictors_df = stock_df.shift().dropna()
    #target_df = stock_df[['Close']].iloc[1:]
    predictors_df = stock_df
    target_df = stock_df[['Close']].shift(-1)
    return(predictors_df,target_df)

X,y = predictors_target(df)

display(X.tail())
display(y.tail())
    



,Open,High,Low,Close,Adj Close,Volume,avg_price,daily_returns,volatility_21,sma_21,...,rsi_14,ema_12,ema_26,MACD_signal,BB_lower,BB_middle,BB_upper,BB%B,OBV_in_million,adx_14
Date,,,,,,,,,,,,,,,,,,,,,
2024-03-11,250.220001,251.479996,247.289993,249.869995,249.869995,1539600,249.714996,-0.004343,2.942407,251.153333,...,50.282001,1.463303,-0.753955,2.217258,247.767808,250.913998,254.060188,0.334085,259.6658,15.571718
2024-03-12,249.419998,252.179993,249.220001,251.179993,251.179993,1633900,250.499996,0.005243,2.916436,251.235713,...,53.024318,1.340817,-0.701153,2.041970,248.757843,250.409998,252.062152,0.733028,261.2997,14.815413
2024-03-13,250.259995,250.779999,248.460007,249.250000,249.250000,2129000,249.687500,-0.007684,2.821164,251.323809,...,48.757398,1.075613,-0.773085,1.848698,248.317124,250.059998,251.802871,0.267626,259.1707,14.368420
2024-03-14,247.429993,247.979996,243.600006,246.059998,246.059998,2519700,246.267498,-0.012798,2.776957,251.345237,...,42.648496,0.601101,-0.998078,1.599179,245.778293,249.463998,253.149704,0.038216,256.6510,15.297208
2024-03-15,243.449997,246.380005,243.229996,244.130005,244.130005,3258900,244.297501,-0.007844,3.186383,251.060476,...,39.429686,0.068523,-1.224525,1.293048,242.893063,248.097998,253.302933,0.118824,253.3921,16.245715


,Close
Date,
2024-03-11,251.179993
2024-03-12,249.250000
2024-03-13,246.059998
2024-03-14,244.130005
2024-03-15,NaN


In [81]:
# Data Normalizing & Train-Test Split

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

def train_test_split_scaled(predictors_df, target_df):
    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = X.iloc[21*33*i:(21*33*i)+(21*(33-test_range)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train = target_df.drop(index = predictors_test.index)
    target_test = target_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)
    
    return(predictors_train_scaled,predictors_test_scaled,target_train,target_test)

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split_scaled(X,y)


In [93]:
# Predicting Stock Price using Linear Models
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge


def linear_models_prediction(predictors_train_scaled,predictors_test_scaled,
                             target_train,target_test):
    list_of_models = [Ridge(),Lasso(),MultiTaskElasticNet(),
                  LassoLars(),BayesianRidge()]
    target_predictions_df = target_test.copy()
    for model in list_of_models:
        model_instance = model
        model_instance.fit(predictors_train_scaled,target_train)
        model_prediction = model_instance.predict(predictors_test_scaled)
        target_predictions_df[f'{model}_Close'] = model_prediction

    return(target_predictions_df)

y_predictions = linear_models_prediction(X_train_scaled, X_test_scaled, y_train, y_test)

y_predictions
    

/Users/venkataaditya/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.431e+03, tolerance: 6.229e+02
  model = cd_fast.enet_coordinate_descent(
/Users/venkataaditya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Close,Ridge()_Close,Lasso()_Close,MultiTaskElasticNet()_Close,LassoLars()_Close,BayesianRidge()_Close
Date,,,,,,
2013-08-29,76.769997,77.061590,77.825435,79.411492,77.916313,77.127533
2013-08-30,76.870003,76.835379,77.926269,79.214216,77.695322,76.959086
2013-09-03,77.654999,77.010227,78.066348,79.200156,77.812633,77.084878
2013-09-04,78.059998,78.043895,78.099741,80.140053,78.431913,77.910602
2013-09-05,77.459999,78.452803,78.628275,80.553008,78.848235,78.375912
...,...,...,...,...,...,...
2024-03-11,251.179993,250.656073,247.389479,245.542768,247.508686,249.988056
2024-03-12,249.250000,251.992027,247.719547,246.205158,248.687103,251.436175
2024-03-13,246.059998,250.253841,247.012956,244.868327,246.928611,249.522926


In [91]:
# Plot Predictions using Linear Models

y_predictions.hvplot()



:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [ ]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)